## Libraries & settings

In [14]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver
!pip install beautifulsoup4

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
#chrome_options.add_argument('useAutomationExtention', False)
chrome_options.add_argument('--ifnore-certificate-errors')
chrome_options.add_argument('--start-maximized')

wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/graph

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: use options instead of chrome_options
  from ipykernel import kernelapp as app


In [15]:
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from bs4 import BeautifulSoup 
import pandas as pd 
import os
import time

# Scrapping ebay


In [3]:
#user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:87.0) Gecko/20100101 Firefox/87.0'
#FireFoxDriverPath = os.path.join(os.getcwd(), 'Drivers', 'geckodriver.exe')
#FireFoxProfile = webdriver.FirefoxProfile()
#FireFoxProfile.set_preference("general.useragent.override", user_agent)
url = "https://www.ebay.com/"
browser = wd
browser.implicitly_wait(7)
browser.get(url)

search_keyword = 'Playstation 5'
search_field = browser.find_element_by_id('gh-ac')
search_field.clear()
search_field.send_keys(search_keyword)
search_field.send_keys(Keys.ENTER)

total_result = int(browser.find_element_by_class_name('srp-controls__count-heading').find_element_by_class_name('BOLD').text.replace(',', ''))
ebay_listings = []

current_page = 1
to_continue = True

In [17]:
while to_continue:
    try:
        print('Processing page {0}'.format(current_page))

        soup = BeautifulSoup(browser.page_source, 'html.parser')
        item_list = soup.find_all('li', 's-item--watch-at-corner')
        
        for listing in item_list:
            product_detail = {}

            product_detail['product title'] = listing.h3.text
            product_detail['product url'] = listing.a['href']
            
            listing_subtitles = listing.find_all('div', 's-item__subtitle')
            if len(listing_subtitles) == 2:
                listing_subtitle_1 = listing_subtitles[0].text
                listing_subtitle_2 = listing_subtitles[1].text
            elif len(listing_subtitles) == 1:
                listing_subtitle_1 = listing_subtitles[0].text
                listing_subtitle_2 = None
            else:
                listing_subtitle_1 = None
                listing_subtitle_2 = None

            product_detail['subtitle1'] = listing_subtitle_1
            product_detail['subtitle2'] = listing_subtitle_2


            # Star rating and product review count
            try:
                x_star_rating = listing.find('div', 'x-star-rating')
                stars = len(x_star_rating.find_all('svg', 'icon--star-full-small'))
                if int(stars) > 4: print(stars) 
                review_count = int(listing.find('span', 's-item__reviews-count').span.text.replace( ' product ratings', ''))
                
                product_detail['stars'] = stars
                product_detail['product review_count'] = review_count
            except Exception as e:
                product_detail['stars'] = ''
                product_detail['product review_count'] = ''

            product_detail['price'] = listing.find('span', 's-item__price').text

            try:
                product_detail['shipping detail'] = listing.find('span', 's-item__logisticsCost').text
            except AttributeError:
                product_detail['shipping detail'] = 'Not Available'
                
            ebay_listings.append(product_detail)


        time.sleep(1)
        browser.find_element_by_xpath('//a[@class="pagination__item" and text()="{0}"]'.format(current_page+1)).click()
        current_page +=1

        if current_page > 5:
            raise Exception('Stop')

    except NoSuchElementException:
        print('Last page {0}'.format(current_page))
        to_continue = False



In [18]:
df = pd.DataFrame(ebay_listings)
df.head()

,product title,product url,subtitle1,subtitle2,stars,product review_count,price,shipping detail
0,Sony PlayStation 5 PS5 Console Disc Version *N...,https://www.ebay.com/itm/333896636458?epid=190...,Brand New · Sony PlayStation 5 · Sony PlayStat...,None,0,1976,"$1,049.00",Not Available
1,PS5 Sony PlayStation 5 Console Disc Version,https://www.ebay.com/itm/324483721021?epid=190...,Pre-Owned · Sony PlayStation 5 · Sony PlayStat...,None,0,1976,$859.99,Free shipping
2,Sony PlayStation 5 Console Disc Version (PS5) ...,https://www.ebay.com/itm/144223680923?epid=190...,Brand New · Sony PlayStation 5 · Sony PlayStat...,None,0,1976,$949.99,Free shipping
3,Sony PlayStation 5 PS5 Console Disc Version Bl...,https://www.ebay.com/itm/193955663603?epid=190...,Brand New · Sony PlayStation 5 · Sony PlayStat...,None,0,1976,$899.99,Free shipping
4,New ListingSony PlayStation 5 PS5 DISC Edition...,https://www.ebay.com/itm/203647656449?hash=ite...,Brand New · Sony PlayStation 5,None,,,$760.00,+$23.77 shipping


In [19]:
df['amount'] = df.price.apply(lambda x: int("".join(filter(str.isdigit, x))))

In [20]:
df.sort_values(by='amount')

,product title,product url,subtitle1,subtitle2,stars,product review_count,price,shipping detail,amount
7,Turtle Beach Elite Atlas Pro Performance Heads...,https://www.ebay.com/itm/324391510282?hash=ite...,None,None,,,$59.95,Free shipping,5995
8,"Razer Kraken Gaming Headset For PC, PS4, PS5, ...",https://www.ebay.com/itm/144123396447?hash=ite...,None,None,,,$59.99,Free shipping,5999
9,SteelSeries Arctis 5 RGB Wired Gaming Headset ...,https://www.ebay.com/itm/334129396663?hash=ite...,None,None,,,$89.95,Free shipping,8995
191,New ListingSony PlayStation 5 PS5 Blu-Ray Edit...,https://www.ebay.com/itm/324827522681?epid=190...,Brand New · Sony PlayStation 5 · Sony PlayStat...,None,0,1976,$102.50,+$77.61 shipping,10250
113,New Listing🎁Sony PlayStation 5 Console🎁Digital...,https://www.ebay.com/itm/265357244952?epid=250...,Brand New · Sony PlayStation 5 · Sony PlayStat...,None,0,461,$202.50,+$50.00 shipping,20250
...,...,...,...,...,...,...,...,...,...
129,New Listingplaystation 5 console,https://www.ebay.com/itm/304180922459?_trkparm...,Brand New · Sony PlayStation 5,None,,,"$1,124.00",+$10.00 shipping,112400
137,Sony PlayStation 5 Console PS5 [Disc Version] ...,https://www.ebay.com/itm/284478117917?epid=190...,Brand New · Sony PlayStation 5,None,0,1976,"$1,151.00",+$30.15 shipping,115100
64,Sony PlayStation 5 PS5 Console Disc Version BR...,https://www.ebay.com/itm/203581141972?epid=190...,Brand New · Sony PlayStation 5 · Sony PlayStat...,None,0,1976,"$1,199.95",Free shipping,119995
103,PS5 Sony Playstation 5 Console Disc Version 🔥B...,https://www.ebay.com/itm/304128005531?epid=190...,Brand New · Sony PlayStation 5 · Sony PlayStat...,None,0,1976,"$1,199.99",Free shipping,119999


In [11]:
#df.to_excel('playstation 5 listings.xlsx', index=False)
print(df.iloc[103])

product title           PS5 Sony Playstation 5 Console Disc Version 🔥B...
product url             https://www.ebay.com/itm/304128005531?epid=190...
subtitle1               Brand New · Sony PlayStation 5 · Sony PlayStat...
subtitle2                                                            None
stars                                                                   0
product review_count                                                 1976
price                                                           $1,199.99
shipping detail                                             Free shipping
amount                                                             119999
Name: 103, dtype: object


In [ ]:
# https://towardsdatascience.com/scraping-multiple-amazon-stores-with-python-5eab811453a8

# scrapping Amazon



In [3]:
def get_url(search_term):
  """Genetare the Url from search term"""
  template = "https://www.amazon.com.mx/s?k={}&__mk_es_MX=%C3%85M%C3%85%C5%BD%C3%95%C3%91&ref=nb_sb_noss_1"
  search_term = search_term.replace(' ', '+')
  return template.format(search_term)

In [4]:
url = get_url('jetson nano')
print(url)

https://www.amazon.com.mx/s?k=jetson+nano&__mk_es_MX=%C3%85M%C3%85%C5%BD%C3%95%C3%91&ref=nb_sb_noss_1


In [5]:
wd.get(url)

In [6]:
# Extract the collection
soup = BeautifulSoup(wd.page_source, 'html.parser')

In [7]:
results = soup.find_all('div', {'data-component-type': 's-search-result'})

In [8]:
len(results)

60

In [9]:
# prototype the record
item = results[0]
atag = item.h2.a
description = atag.text.strip()
url = 'https://www.amazon.com.mx' + atag.get('href')

In [11]:
# price
price_parent = item.find('span', 'a-price')
price = price_parent.find('span', 'a-offscreen').text

In [14]:
# stars
rating = item.i.text

In [19]:
# find the reviews
review_count = item.find('span', {'class':'a-size-base'}).text

In [24]:
# error handling
def extract_record(item):

  # description and url
  atag = item.h2.a
  description = atag.text.strip()
  url = 'https://www.amazon.com.mx' + atag.get('href')

  try:
    # price
    price_parent = item.find('span', 'a-price')
    price = price_parent.find('span', 'a-offscreen').text
  except AttributeError:
    return

  try:
    # reviews and rating
    rating = item.i.text
    review_count = item.find('span', {'class':'a-size-base'}).text
  except AttributeError:
    rating = ''
    review_count = ''

  result = (description, price, rating, review_count, url)

  return result


In [26]:
records = []
results = soup.find_all('div', {'data-component-type': 's-search-result'})

for item in results:
  record = extract_record(item)
  if record:
    records.append(record)

In [28]:
print(records[0])

('NVIDIA Jetson Nano - Kit de desarrollador de 2 GB', '$1,482.99', '4.4 de 5 estrellas', '272', 'https://www.amazon.com.mx/NVIDIA-Jetson-Nano-Kit-desarrollador/dp/B08J157LHH/ref=sr_1_1?__mk_es_MX=%C3%85M%C3%85%C5%BD%C3%95%C3%91&dchild=1&keywords=jetson+nano&qid=1634270630&sr=8-1')


In [5]:
# getting the next page
def get_urls(search_term):
  """Genetare the Url from search term"""
  template = "https://www.amazon.com.mx/s?k={}&__mk_es_MX=%C3%85M%C3%85%C5%BD%C3%95%C3%91&ref=nb_sb_noss_1"
  search_term = search_term.replace(' ', '+')

  # add term query to url
  url = template.format(search_term)

  # add page query placeholder
  url += '&page={}'

  return template.format(search_term)

# error handling
def extract_record(item):

  # description and url
  atag = item.h2.a
  description = atag.text.strip()
  url = 'https://www.amazon.com.mx' + atag.get('href')

  try:
    # price
    price_parent = item.find('span', 'a-price')
    price = price_parent.find('span', 'a-offscreen').text
  except AttributeError:
    return

  try:
    # reviews and rating
    rating = item.i.text
    review_count = item.find('span', {'class':'a-size-base'}).text
  except AttributeError:
    rating = ''
    review_count = ''

  result = (description, price, rating, review_count, url)

  return result

def main(search_term):
  """run maun program"""
  # set webdrriver if needed

  records = []
  url = get_urls(search_term)

  for page in range(1,6):
    wd.get(url.format(page))
    soup = BeautifulSoup(wd.page_source, 'html.parser')
    results = soup.find_all('div', {'data-component-type': 's-search-result'})

    for item in results:
      record = extract_record(item)
      if record:
        records.append(record)
  
  wd.close()

  return pd.DataFrame(records, columns = ['Description', 'Price', 'Rating', 'Reviews', 'Url'])

In [6]:
amazon_scrapping_df = main('jetson nano')

In [8]:
amazon_scrapping_df.head(10)

,Description,Price,Rating,Reviews,Url
0,NVIDIA Jetson Nano - Kit de desarrollador de 2 GB,"$1,482.99",4.4 de 5 estrellas,272,https://www.amazon.com.mx/NVIDIA-Jetson-Nano-K...
1,Módulo de cámara para NVIDIA Jetson Nano Devel...,"$1,019.00",,,https://www.amazon.com.mx/Development-pulgadas...
2,Waveshare Jetson Nano 2GB Developer Kit Packag...,"$3,449.99",4.6 de 5 estrellas,13,https://www.amazon.com.mx/Waveshare-Jetson-Nan...
3,TOP1 Intel AC8265 - NIC de red inalámbrica de ...,$433.32,,,https://www.amazon.com.mx/TOP1-Intel-AC8265-in...
4,Yahboom Carcasa de aleación de aluminio para N...,$585.03,4.0 de 5 estrellas,14,https://www.amazon.com.mx/Yahboom-aleaci%C3%B3...
5,Geekworm N100 - Carcasa de metal para Jetson N...,$499.00,4.3 de 5 estrellas,79,https://www.amazon.com.mx/Geekworm-NVIDIA-Encl...
6,Bicool Power Supply Adapter for Jetson Nano (A...,$317.46,,,https://www.amazon.com.mx/Adaptador-alimentaci...
7,"WENDi NVIDIA Jetson Nano Developer Kit, Small ...","$4,343.46",,,https://www.amazon.com.mx/WENDi-NVIDIA-Develop...
8,Geekworm UPS T208 18650 de 6 celdas (salida má...,"$1,252.93",3.1 de 5 estrellas,3,https://www.amazon.com.mx/Geekworm-tarjeta-exp...
9,Nvidia Jetson Xavier NX Developer Kit (8126740...,"$23,956.00",4.7 de 5 estrellas,120,https://www.amazon.com.mx/nVidia-Jetson-Xavier...


# Scrapping mercado libre

In [ ]:
# Search link
# https://listado.mercadolibre.com.mx/monitor#D[A:monitor]
# Page 2 

In [11]:
def get_url(search_term):
  link = 'https://listado.mercadolibre.com.mx/{}'.format(search_term.replace(' ','-'))
  return link

In [12]:
print(get_url('monitor curvo'))

https://listado.mercadolibre.com.mx/monitor-curvo


In [16]:
url = get_url('monitor curvo')
wd.get(url)
soup = BeautifulSoup(wd.page_source, 'html.parser')

In [21]:
results = soup.find_all('li', {'class': 'ui-search-layout__item'})

In [22]:
print(len(results))

50


In [27]:
print(results[0])

<li class="ui-search-layout__item"><div class="ui-search-result__wrapper"><div class="andes-card andes-card--flat andes-card--default ui-search-result ui-search-result--core andes-card--padding-default"><div class="ui-search-result__image"><a class="ui-search-link" href="https://www.mercadolibre.com.mx/monitor-gamer-curvo-samsung-c32r500-led-32-dark-blue-gray-100v240v/p/MLM17952953?pdp_filters=category:MLM1656#searchVariation=MLM17952953&amp;position=1&amp;search_layout=stack&amp;type=product&amp;tracking_id=798d9a73-0fa3-4406-b2b2-09b4bee387f4" title='Monitor gamer curvo Samsung C32R500 led 32 " dark blue gray 100V/240V'><div class="carousel-container arrow-visible"><div class="slick-initialized slick-slider"><div class="slick-list" role="presentation"><div class="slick-track"><div class="slick-slide slick-active" data-index="0" style="outline:none" tabindex="-1"><img alt='Monitor gamer curvo Samsung C32R500 led 32 " dark blue gray 100V/240V' class="ui-search-result-image__element" he

In [35]:
# prototype the record
item = results[0]
atag = item.h2.text.strip()
#description = atag.text.strip()
#url = 'https://listado.mercadolibre.com.mx/' + atag.get('href')

In [41]:
print(item.a['href'])

https://www.mercadolibre.com.mx/monitor-gamer-curvo-samsung-c32r500-led-32-dark-blue-gray-100v240v/p/MLM17952953?pdp_filters=category:MLM1656#searchVariation=MLM17952953&position=1&search_layout=stack&type=product&tracking_id=798d9a73-0fa3-4406-b2b2-09b4bee387f4
